<a href="https://colab.research.google.com/github/punkmic/pyspark-concepts/blob/master/Search_and_Filter_DataFrames_in_PySpark_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Search and Filter DataFrames in PySpark HW

Now it's time to put what you've learn into action with a homework assignment!

In case you need it again, here is the link to the documentation for the full list available function in pyspark.sql.functions library:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions


### First set up your Spark Session!
Alright so first things first, let's start up our pyspark instance.

In [3]:
!pip install -q pyspark

  Preparing metadata (setup.py) ... done


In [11]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('session').getOrCreate()

In [12]:
spark

In [13]:
# print available cores
spark._jsc.sc().getExecutorMemoryStatus().keySet().size()

1

## Read in the DataFrame for this Notebook

We will be continuing to use the fifa19.csv file for this notebook. Make sure that you are writting the correct path to the file. 

In [15]:
fifa = spark.read.csv('fifa19.csv', inferSchema=True, header=True)

## About this dataframe

The **fifa19.csv** dataset includes a list of all the FIFA 2019 players and their attributes listed below: 

 - **General**: Age, Nationality, Overall, Potential, Club
 - **Metrics:** Value, Wage
 - **Player Descriptive:** Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight
 - **Possition:** LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, 
 - **Other:** Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

**Source:** https://www.kaggle.com/karangadiya/fifa19

Use the .toPandas() method to view the first few lines of the dataset so we know what we are working with. 

In [16]:
fifa.limit(3).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94,27,24,33,9,9,15,15,11,€228.1M


Now print the schema of the dataset so we can see the data types of all the varaibles. 

In [18]:
print(fifa.printSchema())

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nu

## Now let's get started!

### First things first..... import the pyspark sql functions library

Since we know we will be using it a lot.

In [19]:
from pyspark.sql.functions import *

### 1. Select the Name and Position of each player in the dataframe

In [22]:
fifa.select(['Name', 'Position']).show(5, False)

+-----------------+--------+
|Name             |Position|
+-----------------+--------+
|L. Messi         |RF      |
|Cristiano Ronaldo|ST      |
|Neymar Jr        |LW      |
|De Gea           |GK      |
|K. De Bruyne     |RCM     |
+-----------------+--------+
only showing top 5 rows



### 1.1 Display the same results from above sorted by the players names

In [26]:
fifa.select(['Name', 'Position']).orderBy('Name').show(5, False)

+-------------+--------+
|Name         |Position|
+-------------+--------+
|A. Abang     |ST      |
|A. Abdellaoui|LB      |
|A. Abdennour |CB      |
|A. Abdi      |CM      |
|A. Abdu Jaber|ST      |
+-------------+--------+
only showing top 5 rows



### 2. Select only the players who belong to a club begining with FC

In [29]:
fifa.select(['Name', 'Club']).where(fifa.Club.like('FC%')).show(5, False)

+---------------+-----------------+
|Name           |Club             |
+---------------+-----------------+
|L. Messi       |FC Barcelona     |
|L. Suárez      |FC Barcelona     |
|R. Lewandowski |FC Bayern München|
|M. ter Stegen  |FC Barcelona     |
|Sergio Busquets|FC Barcelona     |
+---------------+-----------------+
only showing top 5 rows



In [37]:
fifa.select('Name', 'Club').where(fifa.Club.startswith("FC")).limit(5).toPandas()

,Name,Club
0,L. Messi,FC Barcelona
1,L. Suárez,FC Barcelona
2,R. Lewandowski,FC Bayern München
3,M. ter Stegen,FC Barcelona
4,Sergio Busquets,FC Barcelona


### 3. Who is the oldest player in the dataset and how old are they?

Display only the name and age of the oldest player.

In [45]:
fifa.select('Name', 'Age').sort(desc('Age')).show(1)

+--------+---+
|    Name|Age|
+--------+---+
|O. Pérez| 45|
+--------+---+
only showing top 1 row



In [47]:
fifa.select('Name', 'Age').orderBy(fifa['Age'].desc()).show(1)

+--------+---+
|    Name|Age|
+--------+---+
|O. Pérez| 45|
+--------+---+
only showing top 1 row



In [53]:
fifa.select('Name', 'Age').sort(desc('Age')).show(1)

+--------+---+
|    Name|Age|
+--------+---+
|O. Pérez| 45|
+--------+---+
only showing top 1 row



### 4. Select only the following players from the dataframe:

 - L. Messi
 - Cristiano Ronaldo

In [52]:
fifa.select('Name').where(fifa.Name.like('L. Messi') | fifa.Name.like('Cristiano Ronaldo')).show()

+-----------------+
|             Name|
+-----------------+
|         L. Messi|
|Cristiano Ronaldo|
+-----------------+



In [54]:
fifa[fifa.Name.isin('L. Messi', 'Cristiano Ronaldo')].show()

+---+------+-----------------+---+--------------------+-----------+--------------------+-------+---------+------------+--------------------+-------+-----+-------+--------------+------------------------+---------+-----------+--------------+----------+---------+--------+-------------+------------+-----------+--------------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------------+
|_c0|    ID|             Name|Age|               Photo|Nationality|                Flag|Overall|Potential|        Club|           Club Logo|  Value| Wag

### 5. Can you select the first character from the Release Clause variable which indicates the currency used?

In [56]:
fifa.select('Release Clause', fifa['Release Clause'].substr(1,1)).show(5, False)

+--------------+-------------------------------+
|Release Clause|substring(Release Clause, 1, 1)|
+--------------+-------------------------------+
|€226.5M       |€                              |
|€127.1M       |€                              |
|€228.1M       |€                              |
|€138.6M       |€                              |
|€196.4M       |€                              |
+--------------+-------------------------------+
only showing top 5 rows



### 6. Can you select only the players who are over the age of 40?

In [59]:
fifa.filter('Age > 40').show(5, False)

+-----+------+-----------+---+----------------------------------------------+-----------+------------------------------------+-------+---------+----------------------------+-----------------------------------------------+-----+----+-------+--------------+------------------------+---------+-----------+--------------+---------+---------+--------+-------------+-----------+-----------+--------------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+---------+-------+-------+--------+---------+----------+-------------+-----------+------+---------+---------+-------+--------------+-------------+--------+----------+---------+-------------+----------+--------------+
|_c0  |ID    |Name       |Age|Photo                                         

In [57]:
fifa.select('Name', 'Age').where(fifa['Age'] > 40).show(5, False)

+-----------+---+
|Name       |Age|
+-----------+---+
|J. Villar  |41 |
|B. Nivet   |41 |
|O. Pérez   |45 |
|C. Muñoz   |41 |
|S. Narazaki|42 |
+-----------+---+
only showing top 5 rows



### That's is for now... Great Job!